In [11]:
# use CEASELESS or seastar Kernel on adrien local laptop

import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # => to add in Seastar env
import seastar as ss
from seastar.utils.tools import dotdict

from scipy import interpolate, stats # => to add in Seastar env ?
from scipy.optimize import least_squares

# from seastar.gmfs.doppler import *
# import pdb

In [12]:
level1 = xr.Dataset(
    data_vars=dict(
            CentralWavenumber=( [],270 ),
            CentralFreq=( [], 13.5 * 10**9 ),
            IncidenceAngleImage=( ['across','along','Antenna'], np.full([5,6,4], 30) ),
            AntennaAzimuthImage=(['across', 'along', 'Antenna'],
                           np.stack((np.full([5, 6], 45),
                                     np.full([5, 6], 90),
                                     np.full([5, 6], 90),
                                     np.full([5, 6], 135)
                                     ), axis=-1
                                    )
                           ),
            Polarization=(['across', 'along','Antenna'],
                          np.stack((np.full([5, 6], 'VV'),
                                    np.full([5, 6], 'VV'),
                                    np.full([5, 6], 'HH'),
                                    np.full([5, 6], 'VV')
                                    ), axis=-1
                                   )
                          ),
#             Sigma0=( ['across','along','Antenna'], np.full([9,11,4], 1.01) ),
#             dsig0=( ['across','along','Antenna'], np.full([9,11,4], 0.05) ),
#             RVL=( ['across','along','Antenna'], np.full([9,11,4], 0.5) ),
#             drvl=( ['across','along','Antenna'], np.full([9,11,4], 0.01) ),
        ),
    coords=dict(
            across=np.arange(0,5),
            along=np.arange(0,6),
            Antenna=['Fore','MidV','MidH','Aft'],
        ),
)
level1 = level1.set_coords([
    'CentralWavenumber',
    'CentralFreq',
    'IncidenceAngleImage',
    'AntennaAzimuthImage',
    'Polarization', 
])
level1

<xarray.Dataset>
Dimensions:              (across: 5, along: 6, Antenna: 4)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    *empty*

In [13]:
geo = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['across', 'along'], np.full([5, 6], 10)),
            WindDirection=(['across', 'along'], np.full([5, 6], 150)),
            CurrentVelocity=(['across', 'along'], np.full([5, 6], 1)),
            CurrentDirection=(['across', 'along'], np.full([5, 6], 150)),
        ),
        coords=dict(
            across=np.arange(0, 5),
            along=np.arange(0, 6),
        ),
    )
geo

<xarray.Dataset>
Dimensions:           (across: 5, along: 6)
Coordinates:
  * across            (across) int64 0 1 2 3 4
  * along             (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150

In [14]:
gmf=dotdict({'nrcs': dotdict({'name': 'nscat4ds'})})
gmf['doppler'] = dotdict({'name': 'mouche12'})

In [15]:
level1['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf.nrcs)*1.001

In [16]:
model_rvl_list = [None] * level1.Antenna.size
model_wasv_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
#     model_wasv_list[aa] = ss.gmfs.doppler.compute_wasv(level1.sel(Antenna=ant), geo, gmf=gmf.doppler.name)
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf=gmf.doppler.name)
# level1['WASV'] = xr.concat(model_wasv_list, dim='Antenna')*1.001
level1['RVL'] = xr.concat(model_rvl_list, dim='Antenna')*1.001

In [19]:
noise = level1.drop_vars([var for var in level1.data_vars])
# noise = level1.drop_vars(['Sigma0','dsig0','RVL','drvl'])
noise['Sigma0'] = level1.Sigma0*0.05
noise['RVL'] = level1.RVL*0.05

In [22]:
# [geo['U'], geo['V']] = ss.utils.tools.windSpeedDir2UV(geo.WindSpeed, geo.WindDirection)
# [geo['C_U'], geo['C_V']] = ss.utils.tools.currentVelDir2UV(geo.CurrentVelocity, geo.CurrentDirection)

# Test wind_current_retrieval

In [24]:
sL1 = level1.isel(along=slice(0,2), across=slice(0,2))
sN = noise.isel(along=slice(0,2), across=slice(0,2))  # Pass wind_current_retrieval
ambiguity = 'sort_by_cost' 

In [25]:
l2, lmout = ss.retrieval.level2.wind_current_retrieval(sL1, sN, gmf, ambiguity)

To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
To Be Done find_initial_value
To Be Done - ambiguity_sort_by_cost
To Be Done - solve_ambiguity
> /Users/admartin/OneDrive/OneDrive - NOC/Documents/NOC-OneDrive/workLocal/SEASTAR/seastar/seastar/retrieval/level2.py(65)wind_current_retrieval()
     63 
     64         pdb.set_trace()
---> 65         lmmap = xr.concat(lmoutmap, dim='z')
     66         # lmmap['z'] = level1_stack.z
     67         # lmmap = lmmap.reset_index('z')

ipdb> lmmap = lmmap.set_index(z=list_L1s0)
*** NameError: name 'lmmap' is not defined
ipdb> lmmap = xr.concat(lmoutmap, dim='z')
ipdb> lmmap = lmmap.set_index(z=list_L1s0)
ipdb> sol = lmmap.unstack(dim='z')
ipdb> continue
To be done


In [26]:
l2

<xarray.Dataset>
Dimensions:            (across: 2, along: 2, x_variables: 4)
Coordinates:
  * across             (across) int64 0 1
  * along              (along) int64 0 1
  * x_variables        (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
    CentralFreq        float64 1.35e+10
    CentralWavenumber  int64 270
Data variables:
    x                  (x_variables, across, along) float64 -6.424 ... -0.866
    CurrentU           (across, along) float64 0.9546 0.4084 0.5021 0.4998
    CurrentV           (across, along) float64 -0.09061 -0.9056 -0.8645 -0.866
    WindU              (across, along) float64 -6.424 -4.188 -4.5 -4.503
    WindV              (across, along) float64 3.711 8.29 7.802 7.8
    CurrentVelocity    (across, along) float64 0.9589 0.9934 0.9998 0.9999
    CurrentDirection   (across, along) float64 95.42 155.7 149.9 150.0
    WindSpeed          (across, along) float64 7.418 9.287 9.006 9.006
    WindDirection      (across, along) float64 95.42 155.7 149.9 150.0

In [27]:
lmout

<xarray.Dataset>
Dimensions:              (across: 2, along: 2, x_variables: 4, Observables: 2,
                          Antenna: 4, Ambiguities: 4, fun_variables: 8,
                          extrema: 2)
Coordinates:
  * across               (across) int64 0 1
  * along                (along) int64 0 1
  * x_variables          (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
  * Observables          (Observables) <U6 'sigma0' 'RVL'
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
    fun_variables        (Observables, Antenna) int64 0 1 2 3 4 5 6 7
    CentralFreq          float64 1.35e+10
    CentralWavenumber    int64 270
    AntennaAzimuthImage  (Antenna) int64 45 90 90 135
    Polarization         (Antenna) <U2 'VV' 'VV' 'HH' 'VV'
    IncidenceAngleImage  (Antenna) int64 30 30 30 30
Dimensions without coordinates: Ambiguities, extrema
Data variables: (12/17)
    x                    (Ambiguities, x_variables, across, along) float64 -6...
    active_mask          (Ambiguities, x_variables, across, along) int64 0 ... 0
    grad                 (Ambiguities, x_variables, across, along) float64 -2...
    fun                  (Ambiguities, Observables, Antenna, across, along) float64 ...
    jac                  (Ambiguities, fun_variables, x_variables, across, along) float64 ...
    x0                   (Ambiguities, x_variables, across, along) float64 -6...
    ...                   ...
    message              (Ambiguities, across, along) <U42 '`xtol` terminatio...
    success              (Ambiguities, across, along) bool True True ... True
    method               (across, along) <U3 'trf' 'trf' 'trf' 'trf'
    xtol                 (across, along) float64 0.001 0.001 0.001 0.001
    x_scale              (x_variables, across, along) float64 7.0 7.0 ... 0.5
    bounds               (extrema, x_variables, across, along) int64 -30 ... 5